## Food Reommendation Engine Based on Ingredients and Cuisine

In [8]:
import pandas as pd

In [135]:
df = pd.read_csv('/content/indian_food_dataset.csv')

In [137]:
df.drop('Unnamed: 0',1,inplace=True)

In [138]:
df.rename( columns=lambda x:x.lower(),inplace=True )
df.rename(columns= {'translatedrecipename':"recp_name", 'translatedingredients':"ings"  },inplace=True)

In [139]:
df.head(3)

,recp_name,ings,totaltimeinmins,cuisine,translatedinstructions,url,image-url,ingredient-count
0,Masala Karela Recipe,"1 tablespoon Red Chilli powder,3 tablespoon Gr...",45,Indian,"To begin making the Masala Karela Recipe,de-se...",https://www.archanaskitchen.com/masala-karela-...,https://www.archanaskitchen.com/images/archana...,10
1,Spicy Tomato Rice (Recipe),"2 teaspoon cashew - or peanuts, 1/2 Teaspoon ...",15,South Indian Recipes,"To make tomato puliogere, first cut the tomato...",https://www.archanaskitchen.com/spicy-tomato-r...,https://www.archanaskitchen.com/images/archana...,12
2,Ragi Semiya Upma Recipe - Ragi Millet Vermicel...,"1 Onion - sliced,1 teaspoon White Urad Dal (Sp...",50,South Indian Recipes,"To begin making the Ragi Vermicelli Recipe, fi...",https://www.archanaskitchen.com/ragi-vermicell...,https://www.archanaskitchen.com/images/archana...,12


In [140]:
df.loc[555]["ings"]

'10 Raisins,5 Whole Eggs,300 ml Coconut milk,8 Pistachios - finely chopped,1 tablespoon Cinnamon Powder (Dalchini),6 Cashew nuts - finely chopped,200 grams Jaggery - (Kithul Jaggery/ Goan Jaggery),1 tablespoon Cardamom Powder (Elaichi)'

In [142]:

"""
Recommendation engine is based on Ingredients and Cuisine 
As we want only ingredients so we have to remove  words which not falls under ingredients category

Steps:
Removing Stop words using NLTK libray 
Creating list of Custom stop words then remove them ( tablespoon, teaspoon, cup )
"""

df['ings'][0]

'1 tablespoon Red Chilli powder,3 tablespoon Gram flour (besan),2 teaspoons Cumin seeds (Jeera),1 tablespoon Coriander Powder (Dhania),2 teaspoons Turmeric powder (Haldi),Salt - to taste,1 tablespoon Amchur (Dry Mango Powder),6 Karela (Bitter Gourd/ Pavakkai) - deseeded,Sunflower Oil - as required,1 Onion - thinly sliced'

In [148]:
import nltk
from nltk.corpus import stopwords
import re
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

wordnet = WordNetLemmatizer()

# nltk_stops = stopwords.words('english')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk_stops = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [149]:
## Removing Stop Words and other digits
def remove_sw(value):
    text = str(value)
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    text = text.split()

    ## Removing Stop Words
    new_text = []
    for word in text:
        if word not in nltk_stops:
            lmtz_word = wordnet.lemmatize(word) 
            new_text.append(lmtz_word)

    if len(new_text) == 0:
        return True   
    return new_text

In [150]:
df[df['ings'].apply(remove_sw ) ==True]

,recp_name,ings,totaltimeinmins,cuisine,translatedinstructions,url,image-url,ingredient-count


In [151]:
df['ings'] =  df['ings'].apply(remove_sw )

In [154]:
from ast import  literal_eval as lit
from nltk import FreqDist

# tablespoon teaspoon pinch sliced these word are mostly repeated and these are not ingredients 
# sor we are going to remove them
df['ings'].apply(lambda x: FreqDist(x))

0       {'tablespoon': 4, 'red': 1, 'chilli': 1, 'powd...
1       {'teaspoon': 7, 'cashew': 1, 'peanut': 1, 'mus...
2       {'onion': 1, 'sliced': 1, 'teaspoon': 5, 'whit...
3       {'teaspoon': 5, 'turmeric': 1, 'powder': 2, 'h...
4       {'oil': 1, 'per': 2, 'use': 1, 'tablespoon': 3...
                              ...                        
5933    {'teaspoon': 3, 'rose': 1, 'water': 2, 'ghee':...
5934    {'sprig': 1, 'basil': 1, 'leaf': 1, 'finely': ...
5935    {'cup': 3, 'sugar': 1, 'milk': 3, 'vegan': 2, ...
5936    {'pinch': 2, 'turmeric': 1, 'powder': 1, 'hald...
5937    {'salt': 1, 'required': 2, 'clove': 1, 'laung'...
Name: ings, Length: 5938, dtype: object

In [155]:
## creating Custom stop words

kitchen_utils =  pd.read_html('https://www.webstaurantstore.com/guide/582/measurements-and-conversions-guide.html',flavor='html5lib' )
# kitchen_utils.iloc[:,0].to_list() +kitchen_utils.iloc[:,1].to_list()


In [157]:
scrapped_data = []
pd.DataFrame(kitchen_utils[0].iloc[:,0].to_list() + kitchen_utils[0].iloc[:,1].to_list())[0].apply(remove_sw).apply(lambda x: [ scrapped_data.append(i) for i in x  ]   )                                           
scrapped_data

['pinch',
 'dash',
 'teaspoon',
 'tablespoon',
 'tablespoon',
 'tablespoon',
 'teaspoon',
 'tablespoon',
 'tablespoon',
 'cup',
 'cup',
 'pint',
 'quart',
 'le',
 'teaspoon',
 'tablespoon',
 'fluid',
 'oz',
 'cup',
 'cup',
 'cup',
 'cup',
 'fluid',
 'oz',
 'pint',
 'fluid',
 'oz',
 'quart',
 'fluid',
 'oz',
 'gallon',
 'fluid',
 'oz']

In [158]:
## Custom Stop Words

c_stops1 = ['finely', 'chopped','kg', 'gram','optional', 'medium','small','large', 'hour','minute',
            'tbsp','tsp','chop','tightly','packed', 'half','full','ml','handful','size','hard','soft','pinch','inch','required']


c_stops2 = [ 'taste',  'thinly', 'sliced','required', 'inch' ,'powder','thin','per','use', 'roughly','cooked',
                
                'pilled','chilled','garnishing','mixed','like','homemade','extra','whole','freshly','frying', ]

STOPS = scrapped_data + c_stops1 +c_stops2
STOPS = set(STOPS)               
STOPS

{'chilled',
 'chop',
 'chopped',
 'cooked',
 'cup',
 'dash',
 'extra',
 'finely',
 'fluid',
 'freshly',
 'frying',
 'full',
 'gallon',
 'garnishing',
 'gram',
 'half',
 'handful',
 'hard',
 'homemade',
 'hour',
 'inch',
 'kg',
 'large',
 'le',
 'like',
 'medium',
 'minute',
 'mixed',
 'ml',
 'optional',
 'oz',
 'packed',
 'per',
 'pilled',
 'pinch',
 'pint',
 'powder',
 'quart',
 'required',
 'roughly',
 'size',
 'sliced',
 'small',
 'soft',
 'tablespoon',
 'taste',
 'tbsp',
 'teaspoon',
 'thin',
 'thinly',
 'tightly',
 'tsp',
 'use',
 'whole'}

In [159]:
## Removing common words (not ings)
def Remove_customStops(value):
    new_list = []
    for word in value:
        if word not in STOPS:
            new_list.append(word)

    return new_list


df['ings'].apply(Remove_customStops)

0       [red, chilli, flour, besan, cumin, seed, jeera...
1       [cashew, peanut, mustard, dry, red, chilli, wh...
2       [onion, white, urad, dal, split, green, chilli...
3       [turmeric, haldi, coriander, dhania, seed, dry...
4       [oil, coriander, seed, mustard, ginger, tomato...
                              ...                        
5933    [rose, water, ghee, paneer, cottage, cheese, c...
5934    [sprig, basil, leaf, wheat, bread, crumb, ital...
5935    [sugar, milk, vegan, substitute, ingredient, a...
5936    [turmeric, haldi, mustard, seed, sunflower, oi...
5937    [salt, clove, laung, ghee, black, cardamom, ba...
Name: ings, Length: 5938, dtype: object

In [160]:
## Cleaned Ingredients

df['clean_ings'] = df['ings'].apply(Remove_customStops)

In [161]:
df['clean_ings'].sample(5)

3974    [sugar, confectioner, sugar, powdered, sugar, ...
1968    [turmeric, haldi, sunflower, oil, salt, cumin,...
4503    [clove, garlic, sunflower, oil, cooking, garam...
353     [amchur, dry, mango, mustard, seed, kala, jeer...
4759    [salt, water, ghee, jaggery, milk, divided, ri...
Name: clean_ings, dtype: object

In [162]:
df['alldata']  =  df['cuisine'].apply(lambda x :[x.replace(" ","").lower()] ) + df['clean_ings']

In [164]:
## Corpus
def make_corpus(value):
    corpus = []
    for i in value:
        new_sent = " ".join(i)
        corpus.append(new_sent)

    return corpus    

CORPUS = make_corpus(df['alldata'])

In [165]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
tfid = TfidfVectorizer()
tf_cv = tfid.fit_transform(CORPUS).toarray()

from sklearn.metrics.pairwise import cosine_similarity
simi_tf = cosine_similarity(tf_cv)

In [166]:
# pd.DataFrame(tfid.vocabulary_.items()).sort_values(1, ascending=False )
tfid.vocabulary_

{'indian': 957,
 'red': 1682,
 'chilli': 394,
 'flour': 742,
 'besan': 188,
 'cumin': 538,
 'seed': 1817,
 'jeera': 993,
 'coriander': 489,
 'dhania': 585,
 'turmeric': 2168,
 'haldi': 890,
 'salt': 1768,
 'amchur': 57,
 'dry': 636,
 'mango': 1221,
 'karela': 1038,
 'bitter': 219,
 'gourd': 841,
 'pavakkai': 1485,
 'deseeded': 574,
 'sunflower': 2032,
 'oil': 1398,
 'onion': 1404,
 'southindianrecipes': 1941,
 'cashew': 334,
 'peanut': 1491,
 'mustard': 1341,
 'white': 2262,
 'urad': 2190,
 'dal': 552,
 'chickpea': 386,
 'lentil': 1148,
 'green': 862,
 'asafoetida': 92,
 'bc': 169,
 'belle': 181,
 'bhat': 199,
 'rice': 1704,
 'tomato': 2139,
 'split': 1960,
 'lemon': 1144,
 'juice': 1003,
 'ghee': 806,
 'pea': 1488,
 'matar': 1250,
 'hing': 922,
 'carrot': 332,
 'gajjar': 786,
 'sprig': 1969,
 'curry': 542,
 'leaf': 1134,
 'vermicelli': 2222,
 'noodle': 1377,
 'andhra': 65,
 'sesame': 1841,
 'gingelly': 810,
 'clove': 439,
 'garlic': 795,
 'garam': 792,
 'masala': 1242,
 'slit': 1904,


In [167]:

def recommend_recp(name):
    index = df[df['recp_name'] == name].index[0]
    distances = sorted(list(enumerate(simi_tf[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
#         print(df.iloc[i[0]].translatedrecipename,df.iloc[i[0]].cuisine)
        print(f"{df.iloc[i[0]]['recp_name']  }")
            

In [169]:
name = df.sample()['recp_name'].values[0]
print(name)
print("*"*20)
recommend_recp(name)

Maharashtrian Bakar Bhaji Recipe
********************
Maharashtrian Ambat Goda Dal Recipe
Kolhapuri Masala Recipe (Maharashtrian Masala Chutney Powder)
Bhoplya Cha Bharit Recipe – Maharashtrian Pumpkin Sabzi
Maharashtrian Aunty Recipe
Tangy Tamarind Pumpkin Curry Recipe


In [130]:
## Food Reommendation Engine Based on Ingredients and Cuisine
import joblib
joblib.dump(simi_tf, 'cc_foodrcmdns.pkl' )

['cc_foodrcmdns.pkl']